In [3]:
import networkx as nx
import pandas as pd
import csv        
import numpy as np
import matplotlib.pyplot as plt

In [4]:
class DexPools:
    
    def __init__(self, file_path, dex_name):
        
        self.df_pools = pd.read_csv(file_path)
        self.name = dex_name
    
    def filter_first_blocks(self, pools_inp, block_first, block_last):
        df_pools_out = pools_inp.replace(np.nan, -1)
        df_pools_out = df_pools_out.astype({"FirstBlock":"int","LastBlock":"int"})
        df_pools_out = df_pools_out[df_pools_out.FirstBlock >= block_first]
        df_pools_out = df_pools_out[df_pools_out.FirstBlock <= block_last]
        return df_pools_out
    
    def filter_last_blocks(self, pools_inp, block_first, block_last):
        df_pools_out = pools_inp.replace(np.nan, -1)
        df_pools_out = df_pools_out.astype({"FirstBlock":"int","LastBlock":"int"})
        df_pools_out = df_pools_out[df_pools_out.LastBlock >= block_first]
        df_pools_out = df_pools_out[df_pools_out.LastBlock <= block_last]
        return df_pools_out
    
    def filter_count(self, pools_inp, cnt_min, cnt_max):
        df_pools_out = pools_inp.replace(np.nan, -1)
        df_pools_out = df_pools_out.astype({"CntTransfer":"int"})
        df_pools_out = df_pools_out[df_pools_out.CntTransfer >= cnt_min]
        df_pools_out = df_pools_out[df_pools_out.CntTransfer <= cnt_max]
        return df_pools_out
    
    def find_lost_pools(self):
        df_pools_out = self.filter_count(self.df_pools, 100000, 10000000000)
        df_pools_out = self.filter_last_blocks(df_pools_out, 0, 14500000)
        return df_pools_out
    
    def get_edge_cnt(self, token1, token2):
        df_pools = self.df_pools[self.df_pools.AddressToken0 == token1]
        df_pools = df_pools[df_pools.AddressToken1 == token2]
        if(len(df_pools) == 0):
            return None
        else:
            return df_pools.iloc[0]['CntTransfer']

In [5]:
COLUMN_NAME = ["AddressToken0","AddressToken1"]

class WeightedDexGraph:

    def __init__(self, df_pools, name):
        
        self.crypto_graph = nx.Graph()
        self.name = name
        
        symbol = []
        
        nx.set_node_attributes(self.crypto_graph, symbol, "symbol")

        for row in df_pools.index:
                
            sell_token = df_pools['AddressToken0'][row]
            buy_token = df_pools['AddressToken1'][row]
            
            count_transfer = df_pools['CntTransfer'][row]
            
            self.add_node(self.crypto_graph, buy_token)
            self.add_node(self.crypto_graph, sell_token)
            
            sell_symbol = df_pools['SymbolToken0'][row]
            buy_symbol = df_pools['SymbolToken1'][row]
                        
            self.crypto_graph.nodes[sell_token]["symbol"] = sell_symbol
            self.crypto_graph.nodes[buy_token]["symbol"] = buy_symbol
            
            if(not(self.crypto_graph.has_edge(buy_token, sell_token))):
                self.crypto_graph.add_edge(buy_token, sell_token, weight = count_transfer)
                
    def get_eigen_centrality(self):
        
        eigen_centrality = nx.eigenvector_centrality(self.crypto_graph,weight = 'weight', max_iter = 1000) 
        return eigen_centrality
    
    def get_reversed_sorted_ranks(self, dict_ranks):
        
        dict_ans = dict(sorted(dict_ranks.items(), key = lambda x:-1 * x[1]))
        return dict_ans
    
    def change_ranks_with_symbol(self, dict_ranks):
        
        dict_ans = [(self.crypto_graph.nodes[key]["symbol"], value) 
                        for (key, value) in dict_ranks.items()]
        return dict_ans
    
    def get_top_ranks(self, rank_inp, number_of_tokens):
        return rank_inp[0:number_of_tokens - 1]
        
    def get_page_rank(self):
        
        page_rank = nx.pagerank(self.crypto_graph,weight = 'weight', max_iter = 1000, alpha=0.9)
        return page_rank
    
    
    
    def get_degree(self, node_name):
        if(not self.crypto_graph.has_node(node_name)):
            return 0
        return self.crypto_graph.degree[node_name]

    def add_node(self, graph_input, token_input):
        if(not graph_input.has_node(token_input)):
            graph_input.add_node(token_input)
    
    def get_list_degree(self):
        degree_node_list = self.crypto_graph.degree()
        degree_list = [deg[1] for deg in degree_node_list]
        degree_list.sort()
        return degree_list

    def print(self, file_path_output):
        graph_file = open(file_path_output, 'w')
        graph_writer = csv.writer(graph_file)
        graph_writer.writerow(COLUMN_NAME)
        for edge in self.crypto_graph.edges:
             graph_writer.writerow(edge)

    def get_best_vertices(self, num):
        degree_node_list = self.crypto_graph.degree()
        degree_node = sorted(degree_node_list, key = lambda vertex: vertex[1])    
        return degree_node
    
    def get_count_deg(self, degree_list):
        count_deg = [(num, degree_list.count(num)) for num in degree_list]
        count_deg = [*set(count_deg)]
        count_deg = sorted(count_deg, key = lambda x: x[0])
        return count_deg
    
    def draw_plot_degree(self, degree_list, flag = False):
        
        count_deg = self.get_count_deg(degree_list)
        
        x = np.array([tmp[1] for tmp in count_deg])
        y = -0.8435 * x + 4.6
        print(x, y)

        plt.xscale('log')
        plt.yscale('log')
        plt.plot([tmp[1] for tmp in count_deg], [tmp[0] for tmp in count_deg])
        if(flag):
            plt.xlabel("Log Count Nodes")
            plt.ylabel("Log Degree")
            plt.show()
 
        plt.plot(x, y)
        plt.ylabel("Log Count Nodes")
        plt.xlabel("Log Degree")
        plt.title(f"{self.name} Log Log plot")
        
        plt.show()
    
    def print_info(self, is_avg_clustering = True, is_plot = True, is_page_rank = True,
            is_betweeness = False, is_eigen = False, is_closeness = False):
        
        print(f"Number of Nodes {self.name} is: ", self.crypto_graph.number_of_nodes())
        print(f"Number of Edges {self.name} is: ", self.crypto_graph.number_of_edges())
        print(f"Average Degree {self.name} is: ", (self.crypto_graph.number_of_edges() * 2) 
      / self.crypto_graph.number_of_nodes())
        if(is_avg_clustering):
            print(f"Average Clustering {self.name} is: ", nx.average_clustering(uni_graph.crypto_swaps_graph))
        if(is_plot):
            degree_list_graph = self.get_list_degree()
            self.draw_plot_degree(degree_list_graph)
            
        if(is_page_rank):    
            page_rank = self.get_sorted_page_rank_nodes()
            print(f"{self.name} Top 100 Tokens: ", page_rank[-1:-100:-1])
        
        if(is_betweeness):
            betweenness_centrality = nx.betweenness_centrality(dex_graph.crypto_swaps_graph)
            betweenness_centrality = sorted(betweenness_centrality.items(), key = operator.itemgetter(1))    
            print("Betweenness", betweenness_centrality)
            
        if(is_eigen):
            eigen_vector_centrality = nx.eigenvector_centrality(dex_graph.crypto_swaps_graph)
            print("EigenVector", eigen_vector_centrality)
        
        if(is_closeness):
            closeness_centrality = nx.closeness_centrality(dex_graph.crypto_swaps_graph)
            closeness_centrality = sorted(closeness_centrality.items(), key = operator.itemgetter(1))    
            print("Closeness", closeness_centrality)
            
        print("Components:")
        num_componenent = [x for x in nx.connected_components(self.crypto_graph)]
        print([len(component) for component in num_componenent])
        
        
        num_nodes = self.crypto_graph.number_of_nodes()
        max_edges = (num_nodes * (num_nodes - 1)) / 2
        print("Density of the graph is: ", (self.crypto_graph.number_of_edges() * 2) / (max_edges))
        
    def check_power_law(self):
            
        count_deg = self.get_count_deg(self.get_list_degree())

        x = [np.log(sample[0]) for sample in count_deg]
        y = [np.log(sample[1]) for sample in count_deg]
        x2 = sm.add_constant(x)
        est = sm.OLS(y, x2)
        print(est.fit().summary())
        print(est.fit().f_pvalue)

<h2><span style="color:orange">Making Without Filter Dex Pools</span></h2>

In [6]:
uni_dex = DexPools("../PoolsLiquidities/LiquidityPools/UniSwapV2.csv", "UniSwapV2")
sushi_dex = DexPools("../PoolsLiquidities/LiquidityPools/SushiSwap.csv", "SushiSwap")

<h2><span style="color:orange">Making Without Filter Dex Graphs</span></h2>

In [7]:
uni_graph = WeightedDexGraph(uni_dex.df_pools, "UniSwapV2")
sushi_graph = WeightedDexGraph(sushi_dex.df_pools, "SushiSwap")

<h2><span style="color:orange">Eigen Centrality for Graphs</span></h2>

In [8]:
eigen_vec = uni_graph.get_eigen_centrality()
eigen_vec = uni_graph.get_reversed_sorted_ranks(eigen_vec)
eigen_vec = uni_graph.change_ranks_with_symbol(eigen_vec)
sushi_graph.get_top_ranks(eigen_vec, 20)

[('WETH', 0.6904873158786964),
 ('USDT', 0.4782042143694205),
 ('USDC', 0.4164555328637535),
 ('DAI', 0.16872964767598622),
 ('SHIB', 0.13826876911394761),
 ('SAITAMA', 0.10550838949955167),
 ('UNI', 0.07805884145089223),
 ('HEX', 0.06340173370643366),
 ('KISHU', 0.05600199499508521),
 ('WBTC', 0.05370343993626589),
 ('AMPL', 0.04501505784352522),
 ('LINK', 0.042947142523790595),
 ('ELON', 0.04026161737535216),
 ('eMax', 0.03681353008256764),
 ('STRONG', 0.034265371437017356),
 ('YFI', 0.03002456339119282),
 ('POLS', 0.028979037585452824),
 ('SUSHI', 0.028593870451554204),
 ('SAND', 0.026734756960900122)]

In [9]:
eigen_vec = sushi_graph.get_eigen_centrality()
eigen_vec = sushi_graph.get_reversed_sorted_ranks(eigen_vec)
eigen_vec = sushi_graph.change_ranks_with_symbol(eigen_vec)
sushi_graph.get_top_ranks(eigen_vec, 20)

[('WETH', 0.7037413749462486),
 ('USDC', 0.425593421192498),
 ('USDT', 0.3256757076203685),
 ('SUSHI', 0.23813392477433312),
 ('DAI', 0.2375641940959311),
 ('YFI', 0.09361026135841681),
 ('WBTC', 0.0913129105494755),
 ('CRV', 0.0886448549929251),
 ('SPELL', 0.08830914005297551),
 ('OHM', 0.08628198748065827),
 ('ALCX', 0.08348888866774301),
 ('CVX', 0.07469097344321851),
 ('LINK', 0.06335569629466876),
 ('ILV', 0.05844244692187639),
 ('SNX', 0.051749050364771794),
 ('FTM', 0.04948137178425064),
 ('OHM', 0.047985277322225746),
 ('AAVE', 0.045697237017449335),
 ('COMP', 0.043953753457237996)]

<h2><span style="color:orange">Page Rank Centrality for Graphs</span></h2>

In [13]:
rank_vec = uni_graph.get_page_rank()
rank_vec = uni_graph.get_reversed_sorted_ranks(rank_vec)
rank_vec = uni_graph.change_ranks_with_symbol(rank_vec)
uni_graph.get_top_ranks(rank_vec, 20)

[('WETH', 0.40474512455907413),
 ('USDT', 0.03430883260154094),
 ('USDC', 0.03174279863170742),
 ('DAI', 0.011995307257200523),
 ('SHIB', 0.007006389495330423),
 ('SAITAMA', 0.0052859517439604155),
 ('UNI', 0.0043086263986831345),
 ('WBTC', 0.0037084592052601933),
 ('HEX', 0.003437170952949021),
 ('KISHU', 0.002808218269639916),
 ('AMPL', 0.0023034563828705583),
 ('LINK', 0.0022885502171765865),
 ('ELON', 0.0020242777818097342),
 ('eMax', 0.0018448319636285289),
 ('STRONG', 0.0017536352620413226),
 ('SUSHI', 0.001578720347511097),
 ('YFI', 0.0015554661115289561),
 ('POLS', 0.001456717169537914),
 ('SAND', 0.0014072419581891173)]

In [14]:
rank_vec = sushi_graph.get_page_rank()
rank_vec = sushi_graph.get_reversed_sorted_ranks(rank_vec)
rank_vec = sushi_graph.change_ranks_with_symbol(rank_vec)
sushi_graph.get_top_ranks(rank_vec, 20)

[('WETH', 0.406476824100352),
 ('USDC', 0.049368284610996016),
 ('USDT', 0.04219098231900646),
 ('DAI', 0.0308370571748322),
 ('SUSHI', 0.022121458822418918),
 ('OHM', 0.012736107192170252),
 ('WBTC', 0.011816359292453837),
 ('CRV', 0.008881712205619547),
 ('YFI', 0.00826475797960392),
 ('SPELL', 0.007586624282773023),
 ('ALCX', 0.007168285018674142),
 ('CVX', 0.006391436702085728),
 ('OHM', 0.006246286321384668),
 ('LINK', 0.0057193603681890805),
 ('ILV', 0.0050302505179269895),
 ('SNX', 0.004549446460448045),
 ('FTM', 0.004248649265585473),
 ('AAVE', 0.003985186296549398),
 ('COMP', 0.0037914466566418854)]

<h2><span style="color:orange">Degree With Weights Graphs</span></h2>

In [18]:
degree_list = dict(uni_graph.crypto_graph.degree(weight = 'weight'))
degree_list = uni_graph.get_reversed_sorted_ranks(degree_list)
degree_list = uni_graph.change_ranks_with_symbol(degree_list)
uni_graph.get_top_ranks(degree_list, 20)

[('WETH', 155699678),
 ('USDT', 13769608),
 ('USDC', 13318534),
 ('DAI', 4953335),
 ('SHIB', 2709783),
 ('SAITAMA', 2035974),
 ('UNI', 1700282),
 ('WBTC', 1550031),
 ('HEX', 1412195),
 ('KISHU', 1082028),
 ('LINK', 893424),
 ('AMPL', 891116),
 ('ELON', 780069),
 ('eMax', 709673),
 ('STRONG', 678818),
 ('SUSHI', 618713),
 ('YFI', 603022),
 ('POLS', 562155),
 ('SAND', 548481)]

In [19]:
degree_list = dict(sushi_graph.crypto_graph.degree(weight = 'weight'))
degree_list = sushi_graph.get_reversed_sorted_ranks(degree_list)
degree_list = sushi_graph.change_ranks_with_symbol(degree_list)
sushi_graph.get_top_ranks(degree_list, 20)

[('WETH', 17714282),
 ('USDC', 2256646),
 ('USDT', 1847276),
 ('DAI', 1500893),
 ('SUSHI', 1050087),
 ('OHM', 648386),
 ('WBTC', 589119),
 ('CRV', 436214),
 ('YFI', 398559),
 ('SPELL', 366502),
 ('ALCX', 344734),
 ('OHM', 326262),
 ('CVX', 308400),
 ('LINK', 264800),
 ('ILV', 241954),
 ('SNX', 219506),
 ('FTM', 204329),
 ('AAVE', 191576),
 ('RUNE', 184941)]